In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import Dict, designs, MetalGUI

In [3]:
design = designs.DesignPlanar(overwrite_enabled=True)

In [4]:
gui = MetalGUI(design)

In [5]:
design.variables['cpw_width']='20um'
design.variables['cpw_gap']='12um'

In [8]:
from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

# Res 3GHz e cap 1fF

In [7]:
RouteMeander.get_template_options(design)

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

In [8]:
design.delete_all_components()

In [9]:
cap_opt=Dict(
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='10um',
    cap_gap_ground='12um',
    finger_length='5um',
    finger_count='3',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='5mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='500um',
        end_straight='500um'
    ),
    total_length='20mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [31]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 3GHz cap 1fF.gds')

02:31AM 45s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

In [10]:
from qiskit_metal import renderers

ansys = design.renderers.hfss

In [17]:
ansys.connect_ansys()

INFO 02:48PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:48PM [load_ansys_project]: 	Opened Ansys App
INFO 02:48PM [load_ansys_project]: 	Opened Ansys Desktop v2025.1.0
INFO 02:48PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Desktop/Va-Qiskit/
	Project:   Ressonadores Val
INFO 02:48PM [connect_design]: 	Opened active design
	Design:    res 3GHz cap 1fF [Solution type: Eigenmode]
INFO 02:48PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:48PM [connect]: 	Connected to project "Ressonadores Val" and design "res 3GHz cap 1fF" 😀 



In [13]:
ansys.add_eigenmode_design('res 3GHz cap 1fF')

11:47AM 41s WARNING [add_eigenmode_design]: This method is deprecated. Change your scripts to use new_ansys_design()
11:47AM 42s ERROR [new_ansys_design]: Please install a more recent version of pyEPR (>=0.8.4.4)
INFO 11:47AM [__del__]: Disconnected from Ansys HFSS


AttributeError: 'NoneType' object has no attribute 'GetName'

In [18]:
ansys.clean_active_design()
ansys.render_design(['RES', 'CAP1', 'CAP2'])

In [19]:
ansys.add_eigenmode_setup(
    name='resonance',
    n_modes=2,
    max_passes=15,
    min_converged=2
)

In [19]:
ansys.analyze_setup('resonance')

INFO 02:50PM [get_setup]: 	Opened setup `resonance`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:50PM [analyze]: Analyzing setup resonance


In [20]:
ansys.pinfo.setup.get_solutions().eigenmodes()

([2.96250499693, 5.93602385308], None)

In [21]:
ansys.activate_drivenmodal_design('res 3GHz cap 1fF S21')

03:14PM 33s WARNING [activate_drivenmodal_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
INFO 03:14PM [connect_design]: 	Opened active design
	Design:    res 3GHz cap 1fF S21 [Solution type: HFSS Hybrid Modal Network]
03:14PM 33s WARNING [activate_ansys_design]: The design_name=res 3GHz cap 1fF S21 already exists, but it has solution_type==hfss hybrid modal network, which is different from the requested==drivenmodal. If you want a design with solution type==drivenmodal, please change the name requested for your design to one that does not exist. Alternatively, manually modify the solution_type for design res 3GHz cap 1fF S21 from the Ansys GUI.


In [23]:
ansys.add_drivenmodal_setup(
    name='setup',
    freq_ghz=2.9,
    min_passes=12,
    max_passes=17,
    min_converged=2,
    max_delta_s=0.002,
    basis_order=-1
)

In [22]:
ansys.clean_active_design()
ansys.render_design(selection=['RES', 'CAP1', 'CAP2'], open_pins=[], port_list=[('CAP1', 'south_end',50), ('CAP2', 'south_end',50)])

In [26]:
ansys.disconnect_ansys()

# Res 7GHz cap 1fF

In [6]:
design.delete_all_components()

In [9]:
cap_opt=Dict(
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='10um',
    cap_gap_ground='12um',
    finger_length='5um',
    finger_count='3',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='5mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='1500um',
        end_straight='1500um'
    ),
    total_length='8.3mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [34]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 7GHz cap 1fF.gds')

02:33AM 32s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

In [10]:
from qiskit_metal import renderers

ansys = design.renderers.hfss

In [11]:
ansys.connect_ansys()

INFO 05:24PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:24PM [load_ansys_project]: 	Opened Ansys App
INFO 05:24PM [load_ansys_project]: 	Opened Ansys Desktop v2025.1.0
INFO 05:24PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Desktop/Va-Qiskit/
	Project:   Ressonadores Val
INFO 05:24PM [connect_design]: 	Opened active design
	Design:    res 7GHz cap1fF [Solution type: Eigenmode]
INFO 05:24PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:24PM [connect]: 	Connected to project "Ressonadores Val" and design "res 7GHz cap1fF" 😀 



In [12]:
ansys.activate_eigenmode_design('res 7GHz cap1fF')

05:25PM 00s WARNING [activate_eigenmode_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
INFO 05:25PM [connect_design]: 	Opened active design
	Design:    res 7GHz cap1fF [Solution type: Eigenmode]


In [13]:
ansys.clean_active_design()
ansys.render_design(['RES', 'CAP1','CAP2'])

In [14]:
ansys.add_eigenmode_setup(
    name='resonance',
    n_modes=2,
    max_passes=15,
    min_converged=2
)

In [15]:
ansys.analyze_setup('resonance')

INFO 05:25PM [get_setup]: 	Opened setup `resonance`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:25PM [analyze]: Analyzing setup resonance


In [16]:
ansys.pinfo.setup.get_solutions().eigenmodes()

([7.13176666402, 14.2269598611], None)

In [ ]:
ansys.add_drivenmodal_design('res 7GHz cap 1fF S21')

08:08PM 29s WARNING [add_drivenmodal_design]: This method is deprecated. Change your scripts to use new_ansys_design()
INFO 08:08PM [connect_design]: 	Opened active design
	Design:    res 3GHz cap 1fF S211 [Solution type: HFSS Hybrid Modal Network]
WARNING 08:08PM [connect_setup]: 	No design setup detected.
ERROR 08:08PM [connect_setup]: Original error 😭: 'NoneType' object has no attribute 'name'



Exception:  Did you provide the correct setup name?                            Failed to pull up setup. 😭

In [18]:
ansys.add_drivenmodal_setup(
    name='setup',
    freq_ghz=7.1,
    min_passes=19,
    max_passes=22,
    min_converged=2,
    max_delta_s=0.002,
    basis_order=-1
)

In [19]:
ansys.clean_active_design()
ansys.render_design(selection=['RES', 'CAP1', 'CAP2'], open_pins=[], port_list=[('CAP1', 'south_end',50), ('CAP2', 'south_end',50)])

# Res 13GHz e cap 1fF

In [20]:
design.delete_all_components()

In [21]:
cap_opt=Dict(
    pos_x='1mm',
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='10um',
    cap_gap_ground='12um',
    finger_length='5um',
    finger_count='3',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='4mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='700um',
        end_straight='700um'
    ),
    total_length='4.5mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [37]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 13GHz cap 1fF.gds')

02:35AM 37s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

In [ ]:
ansys.add_eigenmode_design('res 13GHz cap1fF')

10:06PM 56s WARNING [add_eigenmode_design]: This method is deprecated. Change your scripts to use new_ansys_design()
INFO 10:06PM [connect_design]: 	Opened active design
	Design:    res 7GHz cap1fF1 [Solution type: Eigenmode]
WARNING 10:06PM [connect_setup]: 	No design setup detected.
WARNING 10:06PM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:06PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [23]:
ansys.clean_active_design()
ansys.render_design(['RES', 'CAP1','CAP2'])

In [24]:
ansys.add_eigenmode_setup(
    name='resonance',
    n_modes=2,
    max_passes=15,
    min_converged=2
)

In [25]:
ansys.analyze_setup('resonance')

INFO 10:12PM [get_setup]: 	Opened setup `resonance`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:12PM [analyze]: Analyzing setup resonance


In [26]:
ansys.pinfo.setup.get_solutions().eigenmodes()

([13.0007885889, 25.9723609729], None)

In [28]:
ansys.add_drivenmodal_design('res 13GHz cap 1fF S21')

10:22PM 41s WARNING [add_drivenmodal_design]: This method is deprecated. Change your scripts to use new_ansys_design()
INFO 10:22PM [connect_design]: 	Opened active design
	Design:    res 13GHz cap 1fF S21 [Solution type: HFSS Hybrid Modal Network]
WARNING 10:22PM [connect_setup]: 	No design setup detected.
ERROR 10:22PM [connect_setup]: Original error 😭: 'NoneType' object has no attribute 'name'



Exception:  Did you provide the correct setup name?                            Failed to pull up setup. 😭

In [29]:
ansys.add_drivenmodal_setup(
    name='setup',
    freq_ghz=13,
    min_passes=19,
    max_passes=22,
    min_converged=2,
    max_delta_s=0.002,
    basis_order=-1
)

In [30]:
ansys.clean_active_design()
ansys.render_design(selection=['RES', 'CAP1', 'CAP2'], open_pins=[], port_list=[('CAP1', 'south_end',50), ('CAP2', 'south_end',50)])

# Res 3GHz e cap 5fF

In [47]:
design.delete_all_components()

In [48]:
cap_opt=Dict(
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='10um',
    cap_gap_ground='12um',
    finger_length='45um',
    finger_count='3',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='5mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='500um',
        end_straight='500um'
    ),
    total_length='20mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [40]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 3GHz cap 5fF.gds')

02:37AM 18s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

In [34]:
gui.main_window.close()

True

# Res 3GHz e cap 51 fF

In [49]:
design.delete_all_components()

In [50]:
cap_opt=Dict(
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='15um',
    cap_gap='5um',
    cap_gap_ground='12um',
    finger_length='50um',
    finger_count='5',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='5mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='500um',
        end_straight='500um'
    ),
    total_length='20mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [51]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 3GHz cap 51fF.gds')

02:09PM 43s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

# Res 7GHz e cap 51fF

In [52]:
design.delete_all_components()

In [53]:
cap_opt=Dict(
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='15um',
    cap_gap='5um',
    cap_gap_ground='12um',
    finger_length='50um',
    finger_count='5',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='5mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='1500um',
        end_straight='1500um'
    ),
    total_length='8.3mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [54]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 7GHz cap 51fF.gds')

02:11PM 37s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

# Res 13GHz e cap 51fF

In [55]:
design.delete_all_components()

In [56]:
cap_opt=Dict(
    pos_x='1mm',
    orientation='-90',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='15um',
    cap_gap='5um',
    cap_gap_ground='12um',
    finger_length='50um',
    finger_count='5',
    cap_distance='1um'
)

cap1 = CapNInterdigital(design, 'CAP1', options=cap_opt)

cap_opt.orientation='90'
cap_opt.pos_x='4mm'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

res = RouteMeander(design, 'RES', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='north_end'
        ),
        end_pin=Dict(
            component='CAP2',
            pin='north_end'
        )
    ),
    lead=Dict(
        start_straight='700um',
        end_straight='700um'
    ),
    total_length='4.5mm',
    fillet='130um',
    meander=Dict(spacing='280um'),
    hfss_wire_bonds=True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line1 = RouteStraight(design, 'LINE1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP1',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD1',
            pin='tie'
        )
    )
))

line2 = RouteStraight(design, 'LINE2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='CAP2',
            pin='south_end'
        ),
        end_pin=Dict(
            component='PAD2',
            pin='tie'
        )
    )
))

gui.rebuild()
gui.autoscale()

In [57]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['center_x'] = '2.5mm'
design.chips.main.size['center_y'] = '0mm'
design.chips.main

design.renderers.gds.export_to_gds('resonator 13GHz cap 51fF.gds')

02:11PM 56s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"d:\Users\e196203\Desktop\resources".


1

In [58]:
gui.main_window.close()

True